In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install pyodbc
!{sys.executable} -m pip install sqlalchemy

In [2]:
#import libraries\n",

import json
import pandas as pd
import requests
import pyodbc as odbc
from datetime import datetime
from sqlalchemy import create_engine
import sqlalchemy
pd.options.mode.chained_assignment = None

In [3]:
#load json object

order_json_url = 'https://raw.githubusercontent.com/namasteTechnologies/data-analyst-challenge/master/orders.json'

d = json.loads(requests.get(order_json_url).text)

Please enter the DB details in the cell below. Check the Readme for the details about the DB connection. The below connection should work on local MSSQL server(2017) instance.

In [4]:
#Create MS-SQL DB Connection 
#url_db = 'mssql+pyodbc://<username>:<password>@<db_host>/<db_name>?driver=ODBC+Driver+17+for+SQL+Server'
url_db = 'mssql+pyodbc://localhost/NamasteTechDw?driver=ODBC+Driver+17+for+SQL+Server'
engine = sqlalchemy.create_engine(url_db)


In [5]:
# json data into dataframe with customer details

data = pd.json_normalize(d)

data['createddate'] = pd.to_datetime(data['created_at'])

data['datekey'] = data.createddate.apply(lambda x: x.strftime('%Y%m%d')).astype(int)

data['currencykey'] = '2' 

data.head(2)

,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,createddate,datekey,currencykey
0,8369263756632563,79.39,2020-03-07T14:31:11Z,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-07 14:31:11+00:00,20200307,2
1,7262170348080494,86.98,2020-03-01T09:16:30Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-01 09:16:30+00:00,20200301,2


In [6]:
# json data into dataframe with product details

df = pd.json_normalize(d,'line_items', ['id','total_price','created_at'],record_prefix='od_')

df['createddate'] = pd.to_datetime(df['created_at'])

df['datekey'] = df.createddate.apply(lambda x: x.strftime('%Y%m%d')).astype(int)

df['currencykey'] = '2'

df.head(2)

,od_id,od_product_id,od_product_sku,od_product_name,od_price,id,total_price,created_at,createddate,datekey,currencykey
0,610448,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower,29.40,8369263756632563,79.39,2020-03-07T14:31:11Z,2020-03-07 14:31:11+00:00,20200307,2
1,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,8369263756632563,79.39,2020-03-07T14:31:11Z,2020-03-07 14:31:11+00:00,20200307,2


In [7]:
# customer table - dimension

headers = ["CustomerId", "CustomerName", "CustomerEmail"]

df_customer = data [['customer.id', 'customer.name', 'customer.email']].sort_values(by=['customer.id']).drop_duplicates()

df_customer.columns = headers

df_customer.head(2)

,CustomerId,CustomerName,CustomerEmail
17,23955,Daniel Smith,daniel.smith@gmail.com
0,47178,William Doe,william.doe@gmail.com


In [8]:
# product dimension

df_product = df[['od_product_id','od_product_sku','od_product_name']].sort_values(by=['od_product_id']).drop_duplicates()

headers = ["ProductId","ProductSku","ProductName"]

df_product.columns = headers

df_product=df_product.drop_duplicates()

df_product.head(2)

,ProductId,ProductSku,ProductName
33,278266679,GPSTASH-01,Grandpa's Stash Dried Flower
26,477365777,GOGPR-07,Glueberry OG Pre-Rolls


In [9]:
#find exchange rate function to call api to get rate from USD to CAD

def conversion(date):
    
    payload = {'symbols': 'CAD', 'base': 'USD'}
  
    url = 'https://api.exchangeratesapi.io/' + str(date).strip()
  
    r = requests.get(url, params=payload)
    data = r.json()
  
    conversion_rate = data['rates']['CAD']
    
    return(conversion_rate)


In [10]:
# dataframe to store exchange rate

df_orderdates = data[['created_at','createddate','datekey','currencykey']].sort_values(by=['created_at']).drop_duplicates()

df_orderdates['exchangedate'] = pd.to_datetime(df_orderdates['createddate']).dt.date

df_exchange = df_orderdates[['datekey','exchangedate','currencykey']]

df_exchange['exchangerate'] = df_orderdates['exchangedate'].apply(conversion)
df_exchange = df_exchange.drop_duplicates()


In [11]:
#dataframe to store exchangerate

df_exchange.head(2)

,datekey,exchangedate,currencykey,exchangerate
2,20200101,2020-01-01,2,1.299448
14,20200105,2020-01-05,2,1.298197


In [12]:
#orders dataframe

df_orders = data[['id', 'customer.id', 'total_price', 'createddate', 'datekey','currencykey']].sort_values(by=['id']).drop_duplicates()

headers = ["orderid","customerid","total_price", "orderdatetime", "datekey","currencykey"]

df_orders.columns = headers

df_orders.head(2)

,orderid,customerid,total_price,orderdatetime,datekey,currencykey
18,1104846479586545,94720,86.98,2020-02-22 21:44:05+00:00,20200222,2
17,1288301820842508,23955,29.40,2020-01-19 03:53:34+00:00,20200119,2


In [13]:
#Orderdetails dataframe

df_orderdetail =  df[['od_id','id','od_product_id','od_price','datekey','currencykey' ]].sort_values(by=['id']).drop_duplicates()


headers = ["prodorderid","orderid","productid", "saleprice", "datekey","currencykey"]

df_orderdetail.columns = headers

df_orderdetail.head(2)

,prodorderid,orderid,productid,saleprice,datekey,currencykey
31,997208,1104846479586545,278266679,49.99,20200222,2
32,759658,1104846479586545,573794972,36.99,20200222,2


In [14]:
# Insert data into customer table 

df_customer.to_sql('temp_DimCustomer', con=engine, if_exists = 'replace')

connection = engine.connect()


query = "Insert into [DimCustomer] select t.CustomerID, t.CustomerName, t.CustomerEmail \
  from temp_DimCustomer t left join [DimCustomer] d on t.customerid = d.customerid \
  where d.customerid is null \
 drop table temp_DimCustomer" 

connection.execute(query)
connection.close()



In [15]:
# Insert data into Product table 

df_product.to_sql('temp_DimProduct', con=engine, if_exists = 'replace')

connection = engine.connect()


query = "INSERT INTO [DimProduct] SELECT t.ProductID, t.ProductSku, t.ProductName  \
         FROM temp_DimProduct t LEFT JOIN [DimProduct] p ON t.ProductID = p.ProductID \
         WHERE p.ProductID IS NULL \
         DROP TABLE temp_DimProduct"

connection.execute(query)
connection.close()

In [16]:
# Insert data into Exchange Table 

df_exchange.to_sql('temp_FactExchangeRate', con=engine, if_exists = 'replace', dtype = {"exchangedate": sqlalchemy.DateTime()})

connection = engine.connect()


query = "INSERT INTO [FactExchangeRate](DateKey,CurrencyKey,OrderDate,ExchangeRate) SELECT t.DateKey, \
         t.CurrencyKey, t.exchangedate,t.ExchangeRate FROM temp_FactExchangeRate t LEFT JOIN [FactExchangeRate] E \
         ON t.DateKey = E.DateKey and t.CurrencyKey = E.CurrencyKey \
         WHERE (E.CurrencyKey IS NULL OR E.DateKey IS NULL) \
         DROP TABLE temp_FactExchangeRate"

connection.execute(query)
connection.close()

In [17]:
# Insert data into FactOrder Table

df_orders.to_sql('temp_FactOrder', con=engine, if_exists = 'replace',dtype = {"orderdatetime": sqlalchemy.DateTime()})

connection = engine.connect()


query = "INSERT INTO [FactOrder] SELECT t.orderid, t.Customerid,t.total_price,t.orderdatetime,t.datekey,t.currencykey \
         FROM temp_FactOrder t LEFT JOIN [FactOrder] O \
         ON t.orderid = O.OrderID \
         WHERE O.OrderID IS NULL \
         DROP TABLE temp_FactOrder"

connection.execute(query)
connection.close()


In [18]:
# insert data into OrderDetail

df_orderdetail.to_sql('temp_FactOrderDetail', con=engine, if_exists = 'replace')

connection = engine.connect()


query = "INSERT INTO [FactOrderDetail](OrderID,ProductOrderID,ProductID,SalePrice,DateKey,CurrencyKey) \
         SELECT t.orderid, t.prodorderid,t.productid,t.saleprice,t.datekey,t.currencykey \
         FROM temp_FactOrderDetail t LEFT JOIN [FactOrderDetail] OD \
         ON (t.orderid = OD.OrderID and t.productid = OD.productid) \
         WHERE OD.OrderdetailID is null \
         DROP TABLE temp_FactOrderDetail"

connection.execute(query)
connection.close()